In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
train = pd.read_csv("../data/train_labels.csv")
train["name"] = train["name"].apply(lambda x : "../data/train/" + str(x) + ".jpg")
train["invasive"] = train["invasive"].astype("str")

In [3]:
train

,name,invasive
0,../data/train/1.jpg,0
1,../data/train/2.jpg,0
2,../data/train/3.jpg,1
3,../data/train/4.jpg,0
4,../data/train/5.jpg,1
...,...,...
2290,../data/train/2291.jpg,1
2291,../data/train/2292.jpg,1
2292,../data/train/2293.jpg,1
2293,../data/train/2294.jpg,1


In [4]:
from keras.preprocessing.image import ImageDataGenerator
train_image_gen = ImageDataGenerator(rescale = 1/255, validation_split=0.022, rotation_range=270, horizontal_flip=True)
batch_size = 6
image_size = 800
train_generator = train_image_gen.flow_from_dataframe(dataframe=train, batch_size=batch_size, target_size=(image_size,image_size), x_col="name", y_col="invasive", subset="training", class_mode="binary")
val_generator = train_image_gen.flow_from_dataframe(dataframe=train, batch_size=batch_size, target_size=(image_size,image_size), x_col="name", y_col="invasive", subset="validation", class_mode="binary")

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2295 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2295 invalid image filename(s) in x_col="name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [5]:
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet_v2 import MobileNetV2 
model = Sequential()
model.add(InceptionResNetV2(weights="imagenet", include_top=False, pooling="avg", input_shape=(image_size, image_size, 3)))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

219062272/219055592 [==============================] - 3s 0us/step


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 1536)              54336736  
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1537      
Total params: 54,338,273
Trainable params: 54,277,729
Non-trainable params: 60,544
_________________________________________________________________


In [ ]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.0003, decay=2e-7), loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint
modelcheck = ModelCheckpoint("best.h5", save_best_only=True)
model.fit(train_generator, steps_per_epoch=int(np.ceil(train_generator.n/batch_size)), validation_data = val_generator, 
                    validation_steps = int(np.ceil(val_generator.n/batch_size)), epochs=1, callbacks=[modelcheck])

In [ ]:
sub = pd.read_csv("../data/sample_submission.csv")
test = pd.DataFrame({"name": sub["name"].apply(lambda x : "../data/test/" + str(x) + ".jpg")})
test_gen = ImageDataGenerator(rescale = 1./255)
test_generator = test_gen.flow_from_dataframe(dataframe = test, shuffle=False, x_col = "name", y_col = None, class_mode=None, batch_size = batch_size, target_size = (image_size, image_size))
model.load_weights("best.h5")
preds = model.predict_generator(test_generator, steps=int(np.ceil(test_generator.n/batch_size)), workers = 2)
sub["invasive"] = preds
sub.to_csv("submission.csv", index=False)